In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import optuna
from datetime import timedelta

# =========================
# Helper Functions
# =========================

def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  
    dLat = np.radians(lat2 - lat1)
    dLon = np.radians(lat2 - lon1)
    a = np.sin(dLat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dLon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def create_time_features(df):
    df['datetime'] = pd.to_datetime(df['trans_date'] + ' ' + df['trans_time'])
    df['hour'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month
    return df

def create_geographic_features(df):
    df['distance_user_merchant'] = haversine_distance(
        df['lat'], df['long'], df['merch_lat'], df['merch_long']
    )
    return df

def create_user_aggregations(train_df, test_df):
    all_data = pd.concat([train_df, test_df], axis=0, sort=False)
    all_data = all_data.sort_values(by=['cc_num', 'datetime'])
    
    user_trans_count = all_data.groupby('cc_num')['trans_num'].count().rename('user_trans_count')
    user_avg_amt = all_data.groupby('cc_num')['amt'].mean().rename('user_avg_amt')
    user_std_amt = all_data.groupby('cc_num')['amt'].std().rename('user_std_amt').fillna(0)

    all_data = all_data.merge(user_trans_count, on='cc_num', how='left')
    all_data = all_data.merge(user_avg_amt, on='cc_num', how='left')
    all_data = all_data.merge(user_std_amt, on='cc_num', how='left')

    merchant_user_count = all_data.groupby('merchant')['cc_num'].nunique().rename('merchant_user_count')
    all_data = all_data.merge(merchant_user_count, on='merchant', how='left')

    # Time Since Last Transaction
    all_data['time_since_last_trans'] = all_data.groupby('cc_num')['datetime'].diff().dt.total_seconds()
    median_tsl = all_data['time_since_last_trans'].median()
    all_data['time_since_last_trans'].fillna(median_tsl, inplace=True)

    # User hourly patterns
    user_hour_amt = all_data.groupby(['cc_num','hour'])['amt'].mean().rename('user_hour_avg_amt')
    all_data = all_data.join(user_hour_amt, on=['cc_num','hour'])
    all_data['hour_amt_diff'] = all_data['amt'] - all_data['user_hour_avg_amt']
    all_data['hour_amt_diff'].fillna(0, inplace=True)

    train_enriched = all_data[all_data['is_fraud'].notnull()].copy()
    test_enriched = all_data[all_data['is_fraud'].isnull()].copy()

    return train_enriched, test_enriched

def create_amount_percentile_feature(df):
    df['amt_percentile'] = df.groupby('cc_num')['amt'].rank(pct=True)
    return df

def create_fraud_rate_features(train_df, test_df, col):
    fraud_rate = train_df.groupby(col)['is_fraud'].mean().rename(f'{col}_fraud_rate')
    train_df = train_df.merge(fraud_rate, on=col, how='left')
    test_df = test_df.merge(fraud_rate, on=col, how='left')
    overall_fraud = train_df['is_fraud'].mean()
    test_df[f'{col}_fraud_rate'].fillna(overall_fraud, inplace=True)
    return train_df, test_df

def frequency_encoding(train_df, test_df, col):
    freq = train_df[col].value_counts().to_dict()
    train_df[f'{col}_freq'] = train_df[col].map(freq)
    test_df[f'{col}_freq'] = test_df[col].map(freq).fillna(0)
    return train_df, test_df

# =========================
# Main Code
# =========================

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Feature engineering
train = create_time_features(train)
test = create_time_features(test)

train = create_geographic_features(train)
test = create_geographic_features(test)

train = create_amount_percentile_feature(train)
test = create_amount_percentile_feature(test)

# Encode categorical features
cat_features = ['category', 'gender', 'state', 'job', 'merchant', 'city']
for col in cat_features:
    combined = pd.concat([train[[col]], test[[col]]], axis=0)
    le = LabelEncoder()
    le.fit(combined[col].astype(str))
    train[col] = le.transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

# Frequency encoding
for col in ['merchant', 'category']:
    train, test = frequency_encoding(train, test, col)

# Create user-level aggregations and temporal features
train, test = create_user_aggregations(train, test)

# Add fraud rate features for merchant and category
train, test = create_fraud_rate_features(train, test, 'merchant')
train, test = create_fraud_rate_features(train, test, 'category')

# Drop unnecessary columns
drop_cols = [
    'trans_num', 'trans_date', 'trans_time', 'unix_time', 'first', 'last', 
    'street', 'city', 'dob', 'merch_lat', 'merch_long', 'datetime'
]
train.drop(columns=drop_cols, inplace=True, errors='ignore')
test.drop(columns=drop_cols, inplace=True, errors='ignore')

# Separate target
X = train.drop(['is_fraud'], axis=1)
y = train['is_fraud']

test_ids = test['id']
X.drop(columns=['id'], inplace=True, errors='ignore')
test.drop(columns=['id'], inplace=True, errors='ignore')

test = test[X.columns]

# Handle missing values
X.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

# Use StratifiedKFold for CV
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

oof_probs_lgb = np.zeros(len(X))
oof_probs_xgb = np.zeros(len(X))
oof_probs_cat = np.zeros(len(X))

test_probs_lgb_folds = []
test_probs_xgb_folds = []
test_probs_cat_folds = []

# Optional: Hyperparameter Tuning with Optuna for LightGBM (commented as it can be time-consuming)
def objective(trial):
    param = {
        'num_leaves': trial.suggest_int('num_leaves', 31, 128),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'class_weight': 'balanced',
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**param)

    # Obtain the first fold
    train_idx, val_idx = list(skf.split(X, y))[0]
    X_train_o, X_val_o = X.iloc[train_idx], X.iloc[val_idx]
    y_train_o, y_val_o = y.iloc[train_idx], y.iloc[val_idx]

    # Train the model
    model.fit(X_train_o, y_train_o,
              eval_set=[(X_val_o, y_val_o)],
              eval_metric='f1')

    # Predict and calculate F1 score
    val_probs = model.predict_proba(X_val_o)[:, 1]
    best_f1_local = 0
    for t in np.linspace(0.01, 0.99, 99):
        preds = (val_probs >= t).astype(int)
        score = f1_score(y_val_o, preds)
        if score > best_f1_local:
            best_f1_local = score
    return best_f1_local

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)
best_params = study.best_params
print("Best params from Optuna:", best_params)

# For demonstration, use fixed parameters or the found best_params
lgb_params = {
    'n_estimators': 600,
    'learning_rate': 0.05,
    'num_leaves': 63,
    'class_weight': 'balanced',
    'random_state': 42
}

xgb_params = {
    'n_estimators': 600,
    'learning_rate':0.05,
    'max_depth': 7,
    'scale_pos_weight': (len(y)-sum(y))/sum(y),
    'objective':'binary:logistic',
    'random_state':42,
    'use_label_encoder':False,
    'eval_metric':'logloss'
}

cat_params = {
    'iterations':600,
    'learning_rate':0.05,
    'depth':7,
    'auto_class_weights':'Balanced',
    'random_seed':42,
    'eval_metric':'F1',
    'verbose':False
}

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # LightGBM
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  eval_metric='f1')
    val_probs_lgb = lgb_model.predict_proba(X_val_fold)[:,1]
    oof_probs_lgb[val_idx] = val_probs_lgb
    test_probs_lgb = lgb_model.predict_proba(test)[:,1]
    test_probs_lgb_folds.append(test_probs_lgb)

    # XGBoost
    xgb_model = xgb.XGBClassifier(**xgb_params)
    xgb_model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)])
    val_probs_xgb = xgb_model.predict_proba(X_val_fold)[:,1]
    oof_probs_xgb[val_idx] = val_probs_xgb
    test_probs_xgb = xgb_model.predict_proba(test)[:,1]
    test_probs_xgb_folds.append(test_probs_xgb)

    # CatBoost
    cat_model = CatBoostClassifier(**cat_params)
    cat_model.fit(X_train_fold, y_train_fold,
                  eval_set=(X_val_fold, y_val_fold))
    val_probs_cat = cat_model.predict_proba(X_val_fold)[:,1]
    oof_probs_cat[val_idx] = val_probs_cat
    test_probs_cat = cat_model.predict_proba(test)[:,1]
    test_probs_cat_folds.append(test_probs_cat)

# Blend OOF Predictions (simple average)
oof_blended = (oof_probs_lgb + oof_probs_xgb + oof_probs_cat) / 3.0

best_f1 = 0
best_thresh = 0.5
for thresh in np.linspace(0.001,0.99,99):
    val_preds = (oof_blended >= thresh).astype(int)
    score = f1_score(y, val_preds)
    if score > best_f1:
        best_f1 = score
        best_thresh = thresh

print(f"Best Threshold from CV: {best_thresh}, Best OOF F1-score: {best_f1}")

# Blend Test Predictions
test_blended = (np.mean(test_probs_lgb_folds, axis=0) 
                + np.mean(test_probs_xgb_folds, axis=0) 
                + np.mean(test_probs_cat_folds, axis=0)) / 3.0

test_preds = (test_blended >= best_thresh).astype(int)

submission = pd.DataFrame({
    'id': test_ids,
    'is_fraud': test_preds
})

submission.to_csv('my_submission.csv', index=False)
print("Submission saved. Good luck reaching 0.98!")
print(f"Best Threshold from CV: {best_thresh}, Best OOF F1-score: {best_f1}")


/var/folders/gm/mt3gyjgd0q33yy61fn55t56h0000gn/T/ipykernel_48928/3109176742.py:56: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  all_data['time_since_last_trans'].fillna(median_tsl, inplace=True)
/var/folders/gm/mt3gyjgd0q33yy61fn55t56h0000gn/T/ipykernel_48928/3109176742.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

[LightGBM] [Info] Number of positive: 38069, number of negative: 295563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4888
[LightGBM] [Info] Number of data points in the train set: 333632, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


[I 2024-12-08 23:18:01,809] Trial 0 finished with value: 0.9747378455672069 and parameters: {'num_leaves': 85, 'learning_rate': 0.08134719720811782, 'n_estimators': 215, 'min_child_samples': 143, 'subsample': 0.7711979823206199, 'colsample_bytree': 0.7579162144789975}. Best is trial 0 with value: 0.9747378455672069.


[LightGBM] [Info] Number of positive: 38069, number of negative: 295563
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004620 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4888
[LightGBM] [Info] Number of data points in the train set: 333632, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


[I 2024-12-08 23:18:10,452] Trial 1 finished with value: 0.9059040590405905 and parameters: {'num_leaves': 82, 'learning_rate': 0.0013943868160192253, 'n_estimators': 463, 'min_child_samples': 80, 'subsample': 0.6516979761558076, 'colsample_bytree': 0.8690615699949384}. Best is trial 0 with value: 0.9747378455672069.


...

[14]	validation_0-logloss:0.33903
[15]	validation_0-logloss:0.32669
[16]	validation_0-logloss:0.31510
[17]	validation_0-logloss:0.30417
[18]	validation_0-logloss:0.29407
[19]	validation_0-logloss:0.28443
[20]	validation_0-logloss:0.27542
[21]	validation_0-logloss:0.26688
[22]	validation_0-logloss:0.25904
[23]	validation_0-logloss:0.25159
[24]	validation_0-logloss:0.24436
[25]	validation_0-logloss:0.23763
[26]	validation_0-logloss:0.23127
[27]	validation_0-logloss:0.22490
[28]	validation_0-logloss:0.21911
[29]	validation_0-logloss:0.21369
[30]	validation_0-logloss:0.20868
[31]	validation_0-logloss:0.20384
[32]	validation_0-logloss:0.19912
[33]	validation_0-logloss:0.19437
[34]	validation_0-logloss:0.18989
[35]	validation_0-logloss:0.18576
[36]	validation_0-logloss:0.18168
[37]	validation_0-logloss:0.17789
[38]	validation_0-logloss:0.17454
[39]	validation_0-logloss:0.17136
[40]	validation_0-logloss:0.16808
[41]	validation_0-logloss:0.16449
[42]	validation_0-logloss:0.16121
[43]	validatio

/Users/shangweiliu/Library/Python/3.10/lib/python/site-packages/xgboost/core.py:158: UserWarning: [23:29:32] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[14]	validation_0-logloss:0.33967
[15]	validation_0-logloss:0.32725
[16]	validation_0-logloss:0.31572
[17]	validation_0-logloss:0.30470
[18]	validation_0-logloss:0.29450
[19]	validation_0-logloss:0.28484
[20]	validation_0-logloss:0.27580
[21]	validation_0-logloss:0.26724
[22]	validation_0-logloss:0.25927
[23]	validation_0-logloss:0.25169
[24]	validation_0-logloss:0.24458
[25]	validation_0-logloss:0.23784
[26]	validation_0-logloss:0.23153
[27]	validation_0-logloss:0.22556
[28]	validation_0-logloss:0.21969
[29]	validation_0-logloss:0.21438
[30]	validation_0-logloss:0.20919
[31]	validation_0-logloss:0.20437
[32]	validation_0-logloss:0.19949
[33]	validation_0-logloss:0.19483
[34]	validation_0-logloss:0.19048
[35]	validation_0-logloss:0.18636
[36]	validation_0-logloss:0.18241
[37]	validation_0-logloss:0.17852
[38]	validation_0-logloss:0.17487
[39]	validation_0-logloss:0.17147
[40]	validation_0-logloss:0.16833
[41]	validation_0-logloss:0.16510
[42]	validation_0-logloss:0.16068
[43]	validatio

/Users/shangweiliu/Library/Python/3.10/lib/python/site-packages/xgboost/core.py:158: UserWarning: [23:30:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[17]	validation_0-logloss:0.30737
[18]	validation_0-logloss:0.29716
[19]	validation_0-logloss:0.28753
[20]	validation_0-logloss:0.27871
[21]	validation_0-logloss:0.27034
[22]	validation_0-logloss:0.26227
[23]	validation_0-logloss:0.25467
[24]	validation_0-logloss:0.24775
[25]	validation_0-logloss:0.24101
[26]	validation_0-logloss:0.23445
[27]	validation_0-logloss:0.22859
[28]	validation_0-logloss:0.22284
[29]	validation_0-logloss:0.21743
[30]	validation_0-logloss:0.21228
[31]	validation_0-logloss:0.20732
[32]	validation_0-logloss:0.20278
[33]	validation_0-logloss:0.19801
[34]	validation_0-logloss:0.19345
[35]	validation_0-logloss:0.18933
[36]	validation_0-logloss:0.18547
[37]	validation_0-logloss:0.18166
[38]	validation_0-logloss:0.17824
[39]	validation_0-logloss:0.17473
[40]	validation_0-logloss:0.17114
[41]	validation_0-logloss:0.16812
[42]	validation_0-logloss:0.16478
[43]	validation_0-logloss:0.16170
[44]	validation_0-logloss:0.15887
[45]	validation_0-logloss:0.15574
[46]	validatio